# Cargar credenciales de la API
### Desde un archivo .env ubicado en la raíz del proyecto usando load_dotenv(find_dotenv()). De esta forma, como decía la consigna evitamos el poder acceder a CLIENT_ID y CLIENT_SECRET mediante variables de entorno.

In [81]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

## Importación de librerías necesarias


In [83]:
import os
from pathlib import Path
from typing import List, Dict, Union

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())


True

# Se genéra la conexión a la API de Spotify
### Para conectarme con spotify utilice el flujo de credenciales de aplicación con SpotifyClientCredentials, con esto podemos autenticarnos de forma segura y acceder sin necesidad de login del usuario


In [75]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

auth_manager = SpotifyClientCredentials(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)
sp = spotipy.Spotify(auth_manager=auth_manager)


# Funciones para obtener el ID del artista

### Creé dos funciones para obtener el ID del artista, una de ellas que me permitiera extraer el artist_id desde una URL como en el ejemplo o directamente usarlo si ya lo tengo, y la otra busca un artista por nombre y devuelve el primer ID que se encuentre facilitando así la conexión con los datos de Spotify



In [76]:
def normalize_artist_id(text: str) -> str:

    s = text.strip()
    if "/" in s:
        s = s.split("?")[0].rstrip("/")
        return s.split("/")[-1]
    return s

def find_artist_id_by_name(sp_client: spotipy.Spotify, name: str) -> Union[str, None]:

    res = sp_client.search(q=name, type="artist", limit=1)
    items = res.get("artists", {}).get("items", [])
    return items[0]["id"] if items else None


# Con esta función principal nos devuelve el top N
### Creé dos funciones auxiliares para obtener el ID del artista. Una permite extraer el artist_id desde una URL o directamente usarlo si ya lo tengo, y la otra busca un artista por nombre y devuelve el primer ID encontrado, lo que facilita trabajar con entradas más flexibles a la hora de consultar la API de Spotify.

In [77]:

def top_tracks_of_artist(sp_client: spotipy.Spotify,
                         artist_id_or_url_or_name: str,
                         market: str = "US",
                         limit: int = 10) -> List[Dict[str, Union[str, int]]]:

    candidate = normalize_artist_id(artist_id_or_url_or_name)


    if len(candidate) >= 20:
        artist_id = candidate
    else:
        found = find_artist_id_by_name(sp_client, artist_id_or_url_or_name)
        if not found:
            raise ValueError(f"No se encontró artista: {artist_id_or_url_or_name!r}")
        artist_id = found

    resp = sp_client.artist_top_tracks(artist_id, country=market)
    tracks = resp.get("tracks", [])[:limit]

    return [
        {
            "song": t["name"],
            "popularity": int(t.get("popularity", 0)),
            "duration": int(t.get("duration_ms", 0)),
        }
        for t in tracks
    ]


# Generar el top 10 de Beele
### Ejecuté la función principal pasando el nombre del artista y el país como parámetros. Realizando así la lista de las 10 canciones más escuchadas de Beele en este caso

In [80]:
artist_name = "Beele"
MARKET = "US"

top10_beele = top_tracks_of_artist(sp, artist_name, market=MARKET, limit=10)
top10_beele


[{'song': 'La Plena - W Sound 05', 'popularity': 91, 'duration': 150001},
 {'song': 'no tiene sentido', 'popularity': 89, 'duration': 158036},
 {'song': 'YO y TÚ', 'popularity': 90, 'duration': 198153},
 {'song': 'si te pillara', 'popularity': 88, 'duration': 184547},
 {'song': 'mi refe', 'popularity': 85, 'duration': 159404},
 {'song': 'top diesel', 'popularity': 86, 'duration': 189456},
 {'song': 'Hasta Aquí Llegué', 'popularity': 81, 'duration': 188887},
 {'song': 'quédate', 'popularity': 82, 'duration': 176481},
 {'song': 'sobelove', 'popularity': 79, 'duration': 131827},
 {'song': 'Hiekka', 'popularity': 82, 'duration': 213333}]